Importing necessary libraries.

In [1]:
import pandas as pd
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report

Loading and viewing the dataset. This dataset contains 39717 rows and 111 columns.

In [2]:
df = pd.read_csv('/kaggle/input/lending-club-loan-dataset-2007-2011/loan.csv')
df

/tmp/ipykernel_18/602554259.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/lending-club-loan-dataset-2007-2011/loan.csv')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,92187,92174,2500,2500,1075.0,36 months,8.07%,78.42,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39713,90665,90607,8500,8500,875.0,36 months,10.28%,275.38,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39714,90395,90390,5000,5000,1325.0,36 months,8.07%,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39715,90376,89243,5000,5000,650.0,36 months,7.43%,155.38,A,A2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Cleaning the dataframe by dropping the columns that only have null values. Only 57 columns remain.

In [3]:
df.dropna(axis=1, how='all', inplace=True)

df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,Sep-13,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,Apr-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,Jun-16,May-16,0.0,1,INDIVIDUAL,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39712,92187,92174,2500,2500,1075.0,36 months,8.07%,78.42,A,A4,...,NaN,Jun-10,NaN,1,INDIVIDUAL,0,NaN,0,NaN,NaN
39713,90665,90607,8500,8500,875.0,36 months,10.28%,275.38,C,C1,...,NaN,Jul-10,NaN,1,INDIVIDUAL,0,NaN,0,NaN,NaN
39714,90395,90390,5000,5000,1325.0,36 months,8.07%,156.84,A,A4,...,NaN,Jun-07,NaN,1,INDIVIDUAL,0,NaN,0,NaN,NaN
39715,90376,89243,5000,5000,650.0,36 months,7.43%,155.38,A,A2,...,NaN,Jun-07,NaN,1,INDIVIDUAL,0,NaN,0,NaN,NaN


Checking the null count for the remaining columns.

In [4]:
df.isnull().sum()

id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term                              0
int_rate                          0
installment                       0
grade                             0
sub_grade                         0
emp_title                      2459
emp_length                     1075
home_ownership                    0
annual_inc                        0
verification_status               0
issue_d                           0
loan_status                       0
pymnt_plan                        0
url                               0
desc                          12942
purpose                           0
title                            11
zip_code                          0
addr_state                        0
dti                               0
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths              

Dropping the columns where the majority of rows are null. For example, the column `mths_since_last_delinq` has 25682 rows with null values out of the total 39717 rows.

In [5]:
df.drop(columns=['mths_since_last_record', 
                 'next_pymnt_d', 
                 'mths_since_last_delinq'], inplace=True)

Checking the number of unique values for the remaining columns.

In [6]:
unique_counts = df.nunique()
unique_counts

id                            39717
member_id                     39717
loan_amnt                       885
funded_amnt                    1041
funded_amnt_inv                8205
term                              2
int_rate                        371
installment                   15383
grade                             7
sub_grade                        35
emp_title                     28820
emp_length                       11
home_ownership                    5
annual_inc                     5318
verification_status               3
issue_d                          55
loan_status                       3
pymnt_plan                        1
url                           39717
desc                          26526
purpose                          14
title                         19615
zip_code                        823
addr_state                       50
dti                            2868
delinq_2yrs                      11
earliest_cr_line                526
inq_last_6mths              

Dropping the columns where the value in each row is unique. We also drop the columns that have the same value across all rows. Both cases do not display any pattern that can help predict the loan grade of the individual.

In [7]:
columns_to_drop = unique_counts[(unique_counts == df.shape[0]) | (unique_counts == 1)].index
df.drop(columns=columns_to_drop, inplace=True)

Now we must engineer the features to prepare the data for model training.

Our target feature in this dataset is `grade`. This represents the loan grade of the individual. We will use ordinal encoding since this feature has inherent order in its values.

In [8]:
encoder = OrdinalEncoder(categories=[['A', 'B', 'C', 'D', 'E', 'F', 'G']])
encoded_grades = encoder.fit_transform(df[['grade']])
df['grade'] = encoded_grades.astype(int)

Some features have values with string elements such as `%` along with a number. The number portion of the values were extracted.

In [9]:
df['int_rate_'] = df['int_rate'].str.replace('%', '').astype(float)
df['emp_length_'] = df.emp_length.str[:2].str.strip().replace('<', '0').astype(float)
df['revol_util_'] = df.revol_util.str[:-1].astype(float)

Some features were in date format. To convert them to numeric format, the earliest date was taken as the reference and the number of elapsed months was calculated by subtracting the reference date from the current date.

In [10]:
for date_col in ['earliest_cr_line', 'issue_d', 'last_pymnt_d', 'last_credit_pull_d']:
    df[date_col] = pd.to_datetime(df[date_col], format='%b-%y')

# `earliest_cr_line` feature is calculated separately because its earliest date is 1969
elapsed_months = (df['earliest_cr_line'].dt.year - df['earliest_cr_line'].min().year) * 12 + (df['earliest_cr_line'].dt.month - df['earliest_cr_line'].min().month)
df['earliest_cr_line'] = elapsed_months

# Calculating for other date features | January 2007 was selected because the dataset is b/w 2007-2011
earliest_ref_d = pd.to_datetime('2007-01-01')

for date_col in ['issue_d', 'last_pymnt_d', 'last_credit_pull_d']:
    elapsed_months = (df[date_col].dt.year - earliest_ref_d.year) * 12 + (df[date_col].dt.month - earliest_ref_d.month)
    df[date_col] = elapsed_months

The `term` feature only had two possible values, ` 36 months` and ` 60 months`. So it was label encoded.

The `home_ownership` feature was one-hot encoded as it only had three categories.

In [11]:
# Label encoding
df['term_'] = df['term'].map({' 36 months': 0, ' 60 months': 1})

# One-hot encoding
df = pd.get_dummies(df, columns=['home_ownership'])

# Converting the dummy boolean columns to 0 and 1s
boolean_cols = df.select_dtypes(include='bool').columns
df[boolean_cols] = df[boolean_cols].astype(int)

Some features such as `purpose`, `zip_code`, and `addr_state` had a large number of unique categories (high cardinality). If one-hot encoding were to be applied, then we would have a large number of features. This in turn would spread out the data points, making it difficult to find meaningful patterns in data. For this reason, target encoding was used. The mean of the encoded target feature, `grade` in our case, was calculated for each unique category of the above-mentioned features. The mean values were then used for each category, converting the categorical values to numerical values.

In [12]:
# Target encoding
for col in ['purpose', 'zip_code', 'addr_state']:
    df[col] = df[col].map(df.groupby(col)['grade'].mean())

In [13]:
df['loan_status_'] = df['loan_status'].map({'Charged Off': 0,
                                            'Current': 1,
                                            'Fully Paid': 2})
df['verification_status_'] = df['verification_status'].map({'Not Verified': 0,
                                                            'Verified': 1,
                                                            'Source Verified': 2})

Some features with names that ended with `_inv` were removed as they had very similar values with certain other columns.

In [14]:
# Dropping the old columns after the above changes
df.drop(columns=['term',
                 'int_rate',
                 'loan_status',
                 'emp_length',
                 'verification_status',
                 'revol_util',
                 'home_ownership_NONE',   # mostly 0 values
                 'out_prncp',             # ditto
                 'home_ownership_OTHER',  # ditto
                 'desc',                  # irrelevant
                 'earliest_cr_line',      # ditto
                 'title',                 # ditto
                 'emp_title',             # too many categories (28820)
                 'sub_grade',             # an extension of the target feature
                ], inplace=True)

df.drop(columns=df.columns[df.columns.str[-4:] == '_inv'], inplace=True)

Normalization of features is unnecessary since tree-based algorithms can handle the large differences in feature scales. Some features still have missing values. XGBoost can handle them as well.

Now we can split the dataset into training, validation, and test sets with 60-20-20 splits.

In [15]:
X = df.drop('grade', axis=1)
y = df['grade']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

We can train an XGBoost model on the training and validation sets. Validation set is used during training to enable early stopping of the model. This helps prevent overfitting.

After training, we can evaluate the model on the test set to measure its performance on unseen data.

In [16]:
model = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    use_label_encoder=False,
    n_estimators=100,
    early_stopping_rounds=10
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True,
)

[0]	validation_0-mlogloss:1.05611
[1]	validation_0-mlogloss:0.72802
[2]	validation_0-mlogloss:0.52976
[3]	validation_0-mlogloss:0.39417
[4]	validation_0-mlogloss:0.29663
[5]	validation_0-mlogloss:0.22773
[6]	validation_0-mlogloss:0.17591
[7]	validation_0-mlogloss:0.13799
[8]	validation_0-mlogloss:0.10980
[9]	validation_0-mlogloss:0.08945
[10]	validation_0-mlogloss:0.07311
[11]	validation_0-mlogloss:0.06078
[12]	validation_0-mlogloss:0.05169
[13]	validation_0-mlogloss:0.04472
[14]	validation_0-mlogloss:0.03933
[15]	validation_0-mlogloss:0.03482
[16]	validation_0-mlogloss:0.03132
[17]	validation_0-mlogloss:0.02876
[18]	validation_0-mlogloss:0.02670
[19]	validation_0-mlogloss:0.02508
[20]	validation_0-mlogloss:0.02367
[21]	validation_0-mlogloss:0.02247
[22]	validation_0-mlogloss:0.02169
[23]	validation_0-mlogloss:0.02112
[24]	validation_0-mlogloss:0.01990
[25]	validation_0-mlogloss:0.01938
[26]	validation_0-mlogloss:0.01906
[27]	validation_0-mlogloss:0.01845
[28]	validation_0-mlogloss:0.0

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [17]:
y_pred = model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy on test set: 0.9953423967774421
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2017
           1       1.00      1.00      1.00      2404
           2       1.00      1.00      1.00      1620
           3       0.99      0.99      0.99      1062
           4       0.98      0.98      0.98       568
           5       0.97      0.97      0.97       210
           6       0.93      0.90      0.92        63

    accuracy                           1.00      7944
   macro avg       0.98      0.98      0.98      7944
weighted avg       1.00      1.00      1.00      7944



We must tune the hyperparameters to achieve better model performance. We can use random search to search for the optimal hyperparameters.

We rejoined the training and validation sets since the `RandomizedSearchCV` class performs cross-validation.

After finding the optimal hyperparameters, we evaluate the model on the test set.

In [18]:
X_train = pd.concat([X_train, X_val], ignore_index=True)
y_train = pd.concat([y_train, y_val], ignore_index=True)

model = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    use_label_encoder=False
)

param_dist = {
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 6),
    'learning_rate': uniform(0.01, 0.2),
    'n_estimators': randint(100, 500)
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='accuracy',
    cv=5,
    verbose=1,
    random_state=42
)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='mlogloss',
                                           feature_types=None, gamma=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learnin...
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a54aadc34c0>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a54aadc2d40>,
                                        'min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a54aadc13c0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a54adf1a4a0>},
                   random_state=42, scoring='accuracy', verbose=1)

In [19]:
best_params = random_search.best_params_

print(f'Best parameters: {best_params}')
print(f'Best score: {random_search.best_score_}')

y_pred = random_search.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Best parameters: {'learning_rate': 0.11495128632644756, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 148}
Best score: 0.9936109978115224
Accuracy on test set: 0.9950906344410876
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2017
           1       1.00      1.00      1.00      2404
           2       1.00      1.00      1.00      1620
           3       0.99      0.99      0.99      1062
           4       0.98      0.98      0.98       568
           5       0.95      0.97      0.96       210
           6       0.95      0.84      0.89        63

    accuracy                           1.00      7944
   macro avg       0.98      0.97      0.97      7944
weighted avg       1.00      1.00      1.00      7944



Since we have found the optimal hyperparameters, we can use them to initialize a new model. We can train this model on the combined training and validation sets for extra data. The validation set will not be necessary to include since we already know the optimal hyperparameters.

As before, we will evaluate the model on the unseen test set.

We will save the best model.

In [20]:
best_model = XGBClassifier(
    objective='multi:softmax',
    eval_metric='mlogloss',
    use_label_encoder=False,
    **best_params
)

best_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.11495128632644756, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=148, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [21]:
y_pred = best_model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy on test set: 0.9950906344410876
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2017
           1       1.00      1.00      1.00      2404
           2       1.00      1.00      1.00      1620
           3       0.99      0.99      0.99      1062
           4       0.98      0.98      0.98       568
           5       0.95      0.97      0.96       210
           6       0.95      0.84      0.89        63

    accuracy                           1.00      7944
   macro avg       0.98      0.97      0.97      7944
weighted avg       1.00      1.00      1.00      7944



In [22]:
best_model.save_model('best_xgboost_model.json')